## AG-UI와 에이전트 프레임워크 통합 (계속)

### AG-UI 클라이언트 만들기

AG-UI 클라이언트는 원격 서버에 연결하고 스트리밍 응답을 표시합니다. 
다음은 간단한 AG-UI 클라이언트를 만드는 예제입니다.


In [ ]:
"""AG-UI client example."""

import asyncio
import os

from agent_framework import ChatAgent
from agent_framework_ag_ui import AGUIChatClient


async def main():
    """Main client loop."""
    # Get server URL from environment or use default
    server_url = os.environ.get("AGUI_SERVER_URL", "http://127.0.0.1:8888/")
    print(f"AG-UI 서버에 연결 중: {server_url} 🌐\n")

    # Create AG-UI chat client
    chat_client = AGUIChatClient(server_url=server_url)

    # Create agent with the chat client
    agent = ChatAgent(
        name="ClientAgent",
        chat_client=chat_client,
        instructions="당신은 도움이 되는 어시스턴트입니다. 💡✨",
    )

    # Get a thread for conversation continuity
    thread = agent.get_new_thread()

    try:
        while True:
            # Get user input
            message = input("\n사용자 (:q 또는 quit 으로 종료) 💬: ")
            if not message.strip():
                print("요청은 비어있을 수 없습니다. ⚠️")
                continue

            if message.lower() in (":q", "quit"):
                break

            # Stream the agent response
            print("\n어시스턴트 🤖: ", end="", flush=True)
            async for update in agent.run_stream(message, thread=thread):
                # Print text content as it streams
                if update.text:
                    print(f"\033[96m{update.text}\033[0m", end="", flush=True)

            print("\n")

    except KeyboardInterrupt:
        print("\n\n종료 중... 👋")
    except Exception as e:
        print(f"\n\033[91m오류가 발생했습니다: {e} ❌\033[0m")


if __name__ == "__main__":
    asyncio.run(main())